# PL05. Regresión Logística _Scratch_ (con Gradiente Descendente)

__Borja González Seoane. Aprendizaje Automático. Curso 2024-25__



En este *notebook* se procede a implementar un modelo de regresión logística desde cero (_scratch_), es decir, sin utilizar librerías Scikit-Learn. Se implementará el algoritmo de gradiente descendente para minimizar la función de coste.

En este *notebook* se utilizará simplemente un conjunto de datos _dummy_ para probar el modelo. La parte del ejercicio relativa a trabajar con el conjunto de datos Titanic se realizará en el siguiente *notebook* de la PL05, para contrastar también entre diferentes modelos.


In [1]:
import numpy as np
from sklearn.datasets import (  # Interesante para generar datos de prueba
    make_classification,
)

## Creación de datos _dummy_

In [2]:
data = make_classification(
    n_samples = 1000,
    n_features = 20,
)

display(data)
x = data[0]
y = data[1]

(array([[-2.15017405,  0.78458206, -0.16330173, ...,  0.04469772,
          0.8137789 ,  1.03922619],
        [-0.93768773,  0.48820379,  1.64637422, ..., -0.81955796,
         -0.57678974, -0.69439466],
        [ 0.03517578, -0.36665391,  0.04038387, ..., -1.24407626,
         -0.41888296,  1.0266307 ],
        ...,
        [ 1.00983071,  0.43162932, -0.17442641, ...,  1.6352267 ,
         -0.90310221,  1.23752688],
        [ 0.71331689, -1.24903904,  1.15858324, ...,  0.32303203,
          0.75805069, -0.94071444],
        [-1.15180736, -0.68909235, -1.52811751, ..., -0.72661141,
         -1.2500449 , -0.63578947]]),
 array([1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
        1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1,
        0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0,
        1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0,
        1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1,
   

## Implementación del modelo por partes

Primero se procede a implementar las funciones necesarias para el modelo de regresión logística. Se implementarán las siguientes funciones:

1. `predict`: $W^TX + b$, donde $W$ es el vector de pesos y $b$ es el sesgo o _bias_.
2. `sigmoid`: $\frac{1}{1 + e^{-\text{predict}(X)}}$.
3. `loss`: $-\frac{1}{m} \sum_{i=1}^{m} y_i \log(\text{sigmoid}(X_i)) + (1 - y_i) \log(1 - \text{sigmoid}(X_i))$.
4. `dl_dw`, derivada de la función de coste respecto a los pesos: $\frac{1}{m} \sum_{i=1}^{m} (\text{sigmoid}(X_i) - y_i)X_i$.
5. `dl_db`, derivada de la función de coste respecto al sesgo: $\frac{1}{m} \sum_{i=1}^{m} (\text{sigmoid}(X_i) - y_i)$.
6. `update`, actualización de los pesos y el sesgo: $W = W - \alpha \text{dl\_dw}$ y $b = b - \alpha \text{dl\_db}$, siendo $\alpha$ la tasa de aprendizaje.
7. `fit`, función que, a partir de las piezas anteriores, entrena el modelo en un número de iteraciones dado.



In [3]:
def predict(X, w, b):
    """
    Predicción: combinación lineal de los pesos y las características.

    :param X: Datos de entrada.
    :param w: Pesos.
    :param b: Sesgo.
    :return: Predicciones lineales.
    """
    return np.dot(X, w) + b

import numpy as np

def sigmoid(y_hat):
    """
    Función sigmoide para mapear valores a probabilidades en el rango `[0, 1]`.

    :param y_hat: Predicción lineal.
    :return: Valores transformados a probabilidades.
    """
    # Se limitan los valores de `y_hat` para evitar desbordamiento en la exponencial
    y_hat = np.clip(y_hat, -500, 500)

    return 1 / (1 + np.exp(-y_hat))




def loss(y, sigmoid):
    """
    Función de pérdida logística.

    :param y: Etiquetas verdaderas.
    :param sigmoid: Predicciones probabilísticas.
    :return: Pérdida promedio (escalar).
    """
    # Se limitan los valores de `sigmoid` para evitar errores en el cálculo
    sigmoid = np.clip(sigmoid, 1e-15, 1 - 1e-15)

    return -(y * np.log(sigmoid) + (1 - y) * np.log(1 - sigmoid)).mean()


In [4]:
def dldw(X, y, sigmoid):
    """
    Gradiente de la pérdida con respecto a los pesos.

    :param X: Datos de entrada.
    :param y: Etiquetas verdaderas.
    :param sigmoid: Predicciones probabilísticas.
    :return: Gradiente de los pesos.
    """
    return np.dot(X.T, (sigmoid - y)) / X.shape[0]

def dldb(y, sigmoid):
    """
    Gradiente de la pérdida con respecto al sesgo.

    :param y: Etiquetas verdaderas.
    :param sigmoid: Predicciones probabilísticas.
    :return: Gradiente del sesgo.
    """
    return (sigmoid - y).mean()

In [5]:
def update(a, g, lr):
    """
    Actualiza el valor de los pesos o el sesgo usando gradiente descendente.

    :param a: Valor actual.
    :param g: Gradiente.
    :param lr: Tasa de aprendizaje.
    :return: Valor actualizado.
    """
    return a - (g * lr)


def fit(X, y, learning_rate=0.1, n_iter=100, monitorizar_loss=False):
    """
    Ajuste del modelo de regresión logística.

    :param X: Datos de entrada.
    :param y: Etiquetas de clase.
    :param learning_rate: Tasa de aprendizaje.
    :param n_iter: Número de iteraciones.
    :param monitorizar_loss: Si se debe monitorizar la pérdida.
    :return: Pesos y sesgo ajustados.
    """
    # Inicializar los pesos en función del número de características
    w = np.zeros(X.shape[1])
    b = 0

    # Iterar el número de épocas especificado
    for i in range(n_iter):
        # Predicción inicial: combinación lineal de los pesos y las características
        y_hat = predict(X, w, b)

        # Aplica la función sigmoide para obtener probabilidades: mapear a `[0, 1]`
        sig = sigmoid(y_hat)

        # Se calcula la pérdida, en este caso sólo para monitorización, si aplica
        if monitorizar_loss:
            loss_value = loss(y, sig)

            # Imprimir el valor de la pérdida cada 10 iteraciones
            if (i + 1) % 10 == 0:
                print(f"[Iter. {i+1}/{n_iter}] Loss = {loss_value}")

        # Se calculan los gradientes de los pesos y el sesgo
        grad_w = dldw(X, y, sig)
        grad_b = dldb(y, sig)

        # Se actualizan los pesos y el sesgo realizando un paso de gradiente descendente
        w = update(w, grad_w, learning_rate)
        b = update(b, grad_b, learning_rate)

    return w, b



## Arquetipado del modelo completo

Una vez implementadas las funciones anteriores y realizadas algunas pruebas sencillas de corte numérico, se procederá a implementar el modelo completo de regresión logística en forma de clase, con los métodos `fit` y `predict`, siguiendo así el arquetipo habitual que se viene utilizando a lo largo del curso.

In [6]:
import numpy as np


class RegresionLogisticaClassifierScratch:
    def __init__(self, learning_rate=0.1, n_iter=100, monitorizar_loss=False):
        self.learning_rate = learning_rate
        self.n_iter = n_iter
        self.b = 0
        self.w = None  # Se reinicializará en el método `fit`

        self._monitorizar_loss = monitorizar_loss

    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        """
        Ajuste del modelo de regresión logística.

        :param X: Datos de entrada.
        :param y: Etiquetas de clase.
        """
        # Inicializar los pesos en función del número de características. Se inicializa tardíamente porque
        # necesitamos saber el número de características de los datos de entrada, cosa que no sabemos en el
        # constructor
        self.w = np.zeros(X.shape[1])

        # Iterar el número de épocas especificado
        for i in range(self.n_iter):
            # Predicción inicial: combinación lineal de los pesos y las características
            y_hat = self.__predict(X, self.w, self.b)

            # Aplica la función sigmoide para obtener probabilidades: mapear a `[0, 1]`
            sig = self.__sigmoid(y_hat)

            # Se calcula la pérdida, en este caso sólo para monitorización, si aplica
            if self._monitorizar_loss:
                loss = self.__loss(y, sig)

                # Imprimir el valor de la pérdida cada 10 iteraciones
                if (i + 1) % 10 == 0:
                    print(f"[Iter. {i+1}/{self.n_iter}] Loss = {loss}")

            # Se calculan los gradientes de los pesos y el sesgo
            grad_w = self.__dldw(X, y, sig)
            grad_b = self.__dldb(y, sig)

            # Se actualizan los pesos y el sesgo realizando un paso de gradiente descendente
            self.w = self.__update(self.w, grad_w, self.learning_rate)
            self.b = self.__update(self.b, grad_b, self.learning_rate)

    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Predicción de las etiquetas de clase. Se devuelven las etiquetas predichas tras aplicar
        un umbral de 0.5 a las probabilidades.

        :param X: Datos de entrada.
        :return: Etiquetas de clase predichas.
        """
        # Predicción inicial: combinación lineal de los pesos y las características
        y_hat = self.__predict(X, self.w, self.b)

        # Aplica la función sigmoide para obtener probabilidades: mapear a `[0, 1]`
        probs = self.__sigmoid(y_hat)

        # Umbraliza las probabilidades para obtener las etiquetas de clase, que serían 0 o 1
        return (probs >= 0.5).astype(int)

    ###############################################################
    # Se incluyen las funciones desarrolladas en las celdas anteriores como métodos privados
    # de la clase. Se usan métodos estáticos por conveniencia, ya que no se necesita acceder
    # a los atributos de la clase, que se pasan como argumentos a los métodos

    @staticmethod
    def __predict(X, w, b):
        """
        Predicción: combinación lineal de los pesos y las características.

        :param X: Datos de entrada.
        :param w: Pesos.
        :param b: Sesgo.
        :return: Predicciones lineales.
        """
        return np.dot(X, w) + b

    @staticmethod
    def __sigmoid(y_hat):
        """
        Función sigmoide para mapear valores a probabilidades en el rango `[0, 1]`.

        :param y_hat: Predicción lineal.
        :return: Valores transformados a probabilidades.
        """
        # Se limitan los valores de `y_hat` para evitar desbordamiento en la exponencial
        y_hat = np.clip(y_hat, -500, 500)

        return 1 / (1 + np.exp(-y_hat))

    @staticmethod
    def __loss(y, sigmoid):
        """
        Función de pérdida logística.

        :param y: Etiquetas verdaderas.
        :param sigmoid: Predicciones probabilísticas.
        :return: Pérdida promedio (escalar).
        """
        # Se limitan los valores de `sigmoid` para evitar errores en el cálculo
        sigmoid = np.clip(sigmoid, 1e-15, 1 - 1e-15)

        return -(y * np.log(sigmoid) + (1 - y) * np.log(1 - sigmoid)).mean()

    @staticmethod
    def __dldw(X, y, sigmoid):
        """
        Gradiente de la pérdida con respecto a los pesos.

        :param X: Datos de entrada.
        :param y: Etiquetas verdaderas.
        :param sigmoid: Predicciones probabilísticas.
        :return: Gradiente de los pesos.
        """
        return np.dot(X.T, (sigmoid - y)) / X.shape[0]

    @staticmethod
    def __dldb(y, sigmoid):
        """
        Gradiente de la pérdida con respecto al sesgo.

        :param y: Etiquetas verdaderas.
        :param sigmoid: Predicciones probabilísticas.
        :return: Gradiente del sesgo.
        """
        return (sigmoid - y).mean()

    @staticmethod
    def __update(a, g, lr):
        """
        Actualiza el valor de los pesos o el sesgo usando gradiente descendente.

        :param a: Valor actual.
        :param g: Gradiente.
        :param lr: Tasa de aprendizaje.
        :return: Valor actualizado.
        """
        return a - (g * lr)
